# Subspace VQE based on SNAP-displacement for H$_2$ molecule

## Prerequisite

Installation cells for Google Colab users.

In [ ]:
!pip install qutip
!pip install scipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.1/30.1 MB 69.9 MB/s eta 0:00:00


Import libaries.

In [ ]:
import numpy as np
import qutip as qt
import scipy.optimize as sciopt

from functools import partial

In [ ]:
import sys

In [ ]:
import matplotlib.pyplot as plt

## Basics

In [ ]:
def get_cvec_np(r, theta):
    r = np.array(r)
    theta = np.array(theta)
    return r * np.exp(1j * theta)

In [ ]:
def unpack_params_snap_disp(X, nfock):
    # Initialize
    ndepth = X.shape[0] // (nfock + 1)

    # Unpack
    alpha = X[:ndepth].copy()
    d1 = ndepth * nfock
    theta = X[ndepth:ndepth+d1].reshape((ndepth, nfock))

    return alpha, theta


def pack_params_snap_disp(alpha, theta):
    # Initialize
    ndepth = alpha.shape[0]
    nfock = theta.shape[1]
    dim = (nfock + 1) * ndepth
    X = np.zeros((dim,))

    # Pack
    X[:ndepth] = alpha.copy()
    d1 = ndepth * nfock
    X[ndepth:ndepth+d1] = theta.reshape(-1)

    return X

## Ansatz

In [ ]:
def qproj00():
    return qt.basis(2, 0).proj()


def qproj11():
    return qt.basis(2, 1).proj()


def qproj01():
    op = np.array([[0, 1], [0, 0]])
    return qt.Qobj(op)


def qproj10():
    op = np.array([[0, 0], [1, 0]])
    return qt.Qobj(op)


def hadamard():
    op = (1/np.sqrt(2)) * np.array([[1, 1], [1, -1]])
    return qt.Qobj(op)

In [ ]:
def qubit_rot(theta, phi):
    """
    R (theta, phi) = exp[ −i (theta/2) ( X cos(phi) + Y sin(phi) ) ].

    Arguments:
    theta, phi: rotation parameters
    """
    gen = ( qt.sigmax() * np.cos(phi) )
    gen += ( qt.sigmay() * np.sin(phi) )

    H = -1j * (theta / 2) * gen

    return H.expm()

Selective number-dependent arbitray phase (SNAP) and displacement operator ([reference](https://doi.org/10.1103/PhysRevA.92.040303))

\begin{align*}
U (\alpha, \vec{\theta})
&= S (\vec{\theta}) \: D (\alpha),
\\
S (\vec{\theta})
&= \sum_{n = 0}^{L - 1} \: \exp ( i \: \theta_n ) \: |n \rangle \langle n|,
\\
D (\alpha)
&= e^{ \alpha \: ( a^\dagger - a ) }.
\end{align*}

In [ ]:
def snap_disp_op(alpha, thetavec):
    """
    SNAP-displacement operator.

    Arguments:
    alpha -- displacement coefficient
    thetavec -- SNAP parameters
    """
    # Initialize
    nfock = thetavec.shape[0]

    # SNAP
    S2 = np.exp(1j * thetavec[0]) * qt.basis(nfock, 0).proj()
    for i in range(1, nfock):
        S2 += np.exp(1j * thetavec[i]) * qt.basis(nfock, i).proj()

    # Rotation
    D2 = qt.displace(nfock, alpha)

    return S2 * D2

Build the ansatz matrix of depth $N_d$

$$ \mathcal{U} (\vec{\alpha}, \bar{\theta})
= U (\alpha_{N_d}, \vec{\theta}_{N_d}) \cdots
U (\alpha_1, \vec{\theta}_1),
$$

where $\vec{\alpha}$ is an $N_d$-dimensional vector and
$ \bar{\theta}_{N_d \times L} $ is a matrix.


In [ ]:
def snap_disp_ansatz(Xvec, nfock):
    """
    SNAP-displacement ansatz.

    Arguments:
    Xvec -- ansatz parameters
    nfock -- Fock cutoff
    """
    # Initialize
    alphavec, thetamat = unpack_params_snap_disp(Xvec, nfock)
    ndepth = thetamat.shape[0]
    uni = snap_disp_op(alphavec[0], thetamat[0, :])

    # Check
    if ndepth == 1:
        return uni.full()

    # Loop through blocks
    for i in range(1, ndepth):
        new_uni = snap_disp_op(alphavec[i], thetamat[i, :])
        uni = ( new_uni * uni )

    return uni.full()

## Hamiltonian

The two-qubit Hamiltonian ([reference](https://journals.aps.org/prx/supplemental/10.1103/PhysRevX.8.011021/Supplementary.pdf)) is

$$ H_Q
= g_0 (R)
+ g_1 (R) \: Z \otimes I
+ g_2 (R) \: X \otimes X
+ g_3 (R) \: I \otimes Z
+ g_4 (R) \: Z \otimes Z
$$
where $\{ g_j (R) \}$ are real-valued coefficients that depends on the H-H bond distance $R$.

In [ ]:
def exact_ham_h2mol(gvec):
    """
    Hamiltonian.

    Argument:
    gvec -- Hamiltonian coefficients
    """
    H = ( gvec[0] * qt.tensor( qt.qeye(2), qt.qeye(2) ) )
    H += ( gvec[1] * qt.tensor( qt.qeye(2), qt.sigmaz() ) )
    H += ( gvec[2] * qt.tensor( qt.sigmax(), qt.sigmax() ) )
    H += ( gvec[3] * qt.tensor( qt.sigmaz(), qt.qeye(2) ) )
    H += ( gvec[4] * qt.tensor( qt.sigmaz(), qt.sigmaz() ) )

    return H

Hamiltonian coefficients for diffeent bond distances in $Å$.

In [ ]:
hhdis_ref = np.array([0.35, 0.40, 0.45, 0.50, 0.55, 0.65, 0.75, 0.85, 0.95, 1.05,
                      1.15, 1.25, 1.35, 1.45, 1.55, 1.65, 1.75, 1.85, 1.95, 2.05])

# Each slice has (II, IZ, XX, ZI, ZZ)
gmat_hhdis_ref = np.array([[7.01273E-01, -7.47416E-01, 1.62573E-01, -7.47416E-01, 1.31036E-02],
                           [4.60364E-01, -6.88819E-01, 1.64515E-01, -6.88819E-01, 1.29140E-02],
                           [2.67547E-01, -6.33890E-01, 1.66621E-01, -6.33890E-01, 1.27192E-02],
                           [1.10647E-01, -5.83080E-01, 1.68870E-01, -5.83080E-01, 1.25165E-02],
                           [-1.83734E-02, -5.36489E-01, 1.71244E-01, -5.36489E-01, 1.23003E-02],
                           [-2.13932E-01, -4.55433E-01, 1.76318E-01, -4.55433E-01, 1.18019E-02],
                           [-3.49833E-01, -3.88748E-01, 1.81771E-01, -3.88748E-01, 1.11772E-02],
                           [-4.45424E-01, -3.33747E-01, 1.87562E-01, -3.33747E-01, 1.04061E-02],
                           [-5.13548E-01, -2.87796E-01, 1.93650E-01, -2.87796E-01, 9.50345E-03],
                           [-5.62600E-01, -2.48783E-01, 1.99984E-01, -2.48783E-01, 8.50998E-03],
                           [-5.97973E-01, -2.15234E-01, 2.06495E-01, -2.15234E-01, 7.47722E-03],
                           [-6.23223E-01, -1.86173E-01, 2.13102E-01, -1.86173E-01, 6.45563E-03],
                           [-6.40837E-01, -1.60926E-01, 2.19727E-01, -1.60926E-01, 5.48623E-03],
                           [-6.52661E-01, -1.38977E-01, 2.26294E-01, -1.38977E-01, 4.59760E-03],
                           [-6.60117E-01, -1.19894E-01, 2.32740E-01, -1.19894E-01, 3.80558E-03],
                           [-6.64309E-01, -1.03305E-01, 2.39014E-01, -1.03305E-01, 3.11545E-03],
                           [-6.66092E-01, -8.88906E-02, 2.45075E-01, -8.88906E-02, 2.52480E-03],
                           [-6.66126E-01, -7.63712E-02, 2.50896E-01, -7.63712E-02, 2.02647E-03],
                           [-6.64916E-01, -6.55065E-02, 2.56458E-01, -6.55065E-02, 1.61100E-03],
                           [-6.62844E-01, -5.60866E-02, 2.61750E-01, -5.60866E-02, 1.26812E-03]])

Get SNAP displacement ansatz representing $H \otimes I$ and $I \otimes H$, where $H$ is the Hadamard gate.

In [ ]:
def get_xr_snap_two_qubits_nd4(xind):
    """
    SNAP-displacement circuit for H1 and H2 with ndepth = 4.
    """
    # Initialize
    nfock = 4

    # Parameters
    if xind == 1:
        # IH Loss = 5.698748436889549e-16
        Xvec = np.array([ 1.8515008 , -0.74878645,  0.93539459,  1.82961035,  3.57409041,
        2.34920035,  4.32894043, -3.56181614, -0.88322191,  4.56586978,
        3.88532879,  4.17093158,  5.78432709, -0.15713883, -0.57394145,
        0.50031733,  2.01703667,  4.23657671,  2.9336408 , -1.75421509])

    else: # xind == 0
        # HI Loss = 1.0758723404707727e-14
        Xvec = np.array([ 1.66145904,  1.67392585,  0.58363164,  2.31009094,  1.01168818,
       -0.35097461,  2.15607034,  2.02672454,  3.95920415,  0.16670542,
        0.64144534,  1.77308686,  3.21987956, -0.98493528,  1.92483041,
        3.58213862,  3.24853614,  1.03407151,  1.68752729,  0.03674273])

    # Unitary
    U = snap_disp_ansatz(Xvec, nfock)

    return U

## Expectation value

### Probabilities

In [ ]:
def qumode_probs(psi):
    """
    Probabilities.

    Argument:
    psi -- statevector in QuTip
    """
    # Initialize
    probs = []
    nfock = psi.full().shape[0]

    # Results
    for i in range(nfock):
        basis = qt.basis(nfock, i)
        ov = np.abs( psi.overlap(basis) )
        probs.append(ov**2)

    return np.array(probs)

In [ ]:
def qubit_probs(probs, nqubits):
    """
    Converts Fock state probabilities to a dictionary of bitstrings.

    Arguments:
    probs -- List of probabilities for Fock states (output of qumode_probs).
    nqubits -- number of qubits
    """
    # Check
    nfock = len(probs)
    if 2**nqubits != nfock:
        raise ValueError("Dimension mismacth in qubit_probs")

    # Create the dictionary
    bitstring_probs = {}

    for i in range(nfock):
        # Convert the index i to a bitstring of length L
        bitstring = format(i, f'0{nqubits}b')  # Format i as a binary string of length L
        bitstring_probs[bitstring] = probs[i]

    return bitstring_probs

In [ ]:
def pauli_z_exp_val(qubit_prob_dict, pauli_indices):
    """
    <Z(p1) ... Z(pN)>.

    Arguments:
    qubit_prob_dict -- probability dictionary for all basis states
    pauli_indices -- qubit indices for Pauli-Z
    """
    expectation_value = 0

    # Loop over all possible basis states and their probabilities
    for bitstring, prob in qubit_prob_dict.items():
        sign = 1
        # Apply the Pauli operators' sign depending on the state of the qubits
        for idx in pauli_indices:
            if bitstring[idx] == '1':
                sign *= -1
        # Add the weighted probability with the appropriate sign
        expectation_value += sign * prob

    return expectation_value

In [ ]:
def pauli_exp_val_two_qubits(psi, pword):
    """
    <sigma(p1) ... sigma(pN)>. We assume there is no Y in the Pauli words.

    Arguments:
    psi -- two-qubit state
    pword -- Pauli word for two qubits
    """
    # Initialize
    nqubits = 2
    psi_new = qt.Qobj( psi.copy() )
    Zind = []

    # Check
    if len(pword) != nqubits:
        raise ValueError("Dimension mismacth in pauli_exp_val_two_qubits")

    # Rotate
    for i in range(nqubits):
        if pword[i] == 'X':
            U = get_xr_snap_two_qubits_nd4(i)
            psi_new = ( qt.Qobj(U) * psi_new )
            Zind.append(i)
        elif pword[i] == 'Z':
            Zind.append(i)

    # Probabilities
    probs = qumode_probs(psi_new)
    dict_probs = qubit_probs(probs, nqubits)

    # Final
    ov = pauli_z_exp_val(dict_probs, Zind)

    return ov

### Loss function

In [ ]:
def energy_val_basis(Xvec, gvec, nbasis, nfock):
    """
    Compute <psi (n) | H | psi (n)>, where |psi (n)> = U |n>.

    Arguments:
    Xvec -- ansatz parameters
    gvec -- Hamiltonian coefficients
    nbasis -- Fock basis as the initial state
    nfock -- Fock cutoff for the qumode
    """
    # Unitary
    U = snap_disp_ansatz(Xvec, nfock)

    # |psi> = U |n>
    vac = qt.basis(nfock, nbasis).full()
    psi = qt.Qobj(np.matmul(U, vac))

    # Energy
    en = gvec[0].copy()
    en += ( gvec[1] * pauli_exp_val_two_qubits(psi, ['I', 'Z']) )
    en += ( gvec[2] * pauli_exp_val_two_qubits(psi, ['X', 'X']) )
    en += ( gvec[3] * pauli_exp_val_two_qubits(psi, ['Z', 'I']) )
    en += ( gvec[4] * pauli_exp_val_two_qubits(psi, ['Z', 'Z']) )

    return en

In [ ]:
def energy_val_exact(Xvec, gvec, nbasis, nfock):
    """
    Compute <psi (n) | H | psi (n)>, where |psi (n)> = U |n>.

    Arguments:
    Xvec -- ansatz parameters
    gvec -- Hamiltonian coefficients
    nbasis -- Fock basis as the initial state
    nfock -- Fock cutoff for the qumode
    """
    # Unitary
    U = snap_disp_ansatz(Xvec, nfock)

    # |psi> = U |n>
    vac = qt.basis(nfock, nbasis).full()
    psi = qt.Qobj(np.matmul(U, vac))

    # Hamiltonian
    H = exact_ham_h2mol(gvec)

    return qt.expect(qt.Qobj( H.full() ), psi)

In [ ]:
def get_energies(Xvec, gvec, nstates, nfock):
    """
    Energies.

    Arguments:
    Xvec -- ansatz parameters
    gvec -- Hamiltonian coefficients
    nstates -- number of states
    nfock -- Fock cutoff for qumode
    """
    # Check
    if nstates > nfock-1:
        raise ValueError("Wrong nstates in get_energies.")

    # Energies
    envec = []
    for i in range(nstates):
        envec.append( energy_val_basis(Xvec, gvec, i, nfock) )

    return np.array(envec)

In [ ]:
def ssvqe_cost_fun(Xvec, gvec, wvec, nfock):
    """
    Loss function.

    Arguments:
    Xvec -- ansatz parameters
    gvec -- Hamiltonian coefficients
    wvec -- linear cost coefficients
    nfock -- Fock cutoff for qumode
    """
    # Initialize
    nstates = wvec.shape[0]

    # Energies
    envec = get_energies(Xvec, gvec, nstates, nfock)

    # Final
    en = 0.0
    for i in range(nstates):
        en += ( wvec[i] * envec[i] )

    return en

## VQE

In [ ]:
def ssvqe_snap(gvec, wvec, ndepth, maxiter=100, method='COBYLA', verb=0, \
               threshold=1e-08, Xvec=[]):
    """
    Minimize the cost function using SciPy-based methods.

    Arguments:
    gvec -- Hamiltonian coefficients
    wvec -- linear cost coefficients
    nfock -- Fock cutoff for qumode
    ndepth -- ansatz circuit depth
    maxiter -- maximum number of iterations
    method -- optimization method
    verb -- choose additional printing
    threshold -- error tolerance
    Xvec -- optional initial guesses
    """
    # Initialize
    nfock = 4

    # Bound parameters
    alpha_min = 0.0
    alpha_max = 10.0
    theta_min = 0.0
    theta_max = np.pi

    # Define bounds
    bounds = []
    for _ in range(ndepth):
        bounds.append([alpha_min, alpha_max])
    for _ in range(ndepth * nfock):
        bounds.append([theta_min, theta_max])
    bounds = np.array(bounds)

    # Guess
    if len(Xvec) == 0:
        alpha = np.random.uniform(0, 3, size=ndepth)
        theta = np.random.uniform(0, np.pi, size=(ndepth, nfock))
        Xvec = pack_params_snap_disp(alpha, theta)

    # Loss function
    obj_fun = partial(ssvqe_cost_fun, gvec=gvec, wvec=wvec, nfock=nfock)

    # Energy function
    nstates = wvec.shape[0]
    envec_fun = partial(get_energies, gvec=gvec, nstates=nstates, nfock=nfock)

    # Intermediate values
    iteration_step = 0
    print_freq = 10
    def callback(xk):
        nonlocal iteration_step
        iteration_step += 1
        if verb == 1 and (iteration_step % print_freq == 0):
            print("-------------------")
            print(f"iter: {iteration_step}")
            print(f"fval: {envec_fun(xk)}")

    # SciPy options
    options = {'disp': True, 'maxiter': maxiter}

    # Optimize
    if method == 'COBYLA':
        result = sciopt.minimize(obj_fun, Xvec, method=method, bounds=bounds, \
                                 tol=threshold, options=options, callback=callback)
    elif method == 'Powell':
        result = sciopt.minimize(obj_fun, Xvec, method=method, bounds=bounds, \
                                 tol=threshold, options=options, callback=callback)
    elif method == 'CG':
        result = sciopt.minimize(obj_fun, Xvec, method=method, \
                                 tol=threshold, options=options, callback=callback)
    elif method == 'BFGS':
        result = sciopt.minimize(obj_fun, Xvec, method=method, \
                                 tol=threshold, options=options, callback=callback)
    elif method == 'L-BFGS-B':
        result = sciopt.minimize(obj_fun, Xvec, method=method, bounds=bounds, \
                                 tol=threshold, options=options, callback=callback)
    elif method == 'TNC':
        result = sciopt.minimize(obj_fun, Xvec, method=method, \
                                 bounds=bounds, tol=threshold, callback=callback)
    elif method == 'trust-constr':
        result = sciopt.minimize(obj_fun, Xvec, method=method, bounds=bounds, \
                                 tol=threshold, options=options, callback=callback)
    else:
        result = sciopt.minimize(obj_fun, Xvec, method='COBYLA', bounds=bounds, \
                                 tol=threshold, options=options, callback=callback)

    # Final energies
    envec = envec_fun(result.x)

    return envec, result.x, result.nit